In [46]:
# module 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

import copy

In [47]:
plt.rcParams["figure.figsize"] = (14,8)

In [48]:
# dataset 로딩
x_train_load = pd.read_csv('C://Users//zeus_//Desktop//champion_data//train/train_activity_final_v3.csv').sort_values(by='acc_id', ascending=True)
y_train_load = pd.read_csv('C://Users//zeus_//Desktop//champion_data//train/train_label.csv').sort_values(by='acc_id', ascending=True)
x_test_load = pd.read_csv('C://Users//zeus_//Desktop//champion_data//test/test_activity_final_v3.csv').sort_values(by='acc_id', ascending=True)

## 중요 피처 뽑는 함수
- input: data_columns, feature_importances, num
- output: list
---

In [49]:
def select_important_feautre(columns, feature_importance, num):
    
    #딕셔너리
    dic = {}
    for value in [list(v) for v in zip(columns, feature_importance)]:
        dic[value[1]] = value[0] 
        
    sorted_list = sorted(dic)
    sorted_list.reverse()
    
    i = 0
    result = []
    for y in sorted_list:
        result.append(dic[y])
        i = i+1
        if(i == num):
            break
    
    return result

In [50]:
x_data = x_train_load.drop(["Unnamed: 0"], axis=1)
y_data = y_train_load.label
x_test = x_test_load.drop(["Unnamed: 0"], axis=1)

In [51]:
mon2mon_train = pd.read_csv('C://Users//zeus_//Desktop//champion_data/mon2mon/feature148_train.csv')
mon2mon_test = pd.read_csv('C://Users//zeus_//Desktop//champion_data/mon2mon/feature148_test.csv')

In [52]:
x_data = pd.concat([x_data, mon2mon_train.drop(['Unnamed: 0', 'acc_id'], axis = 1)], axis= 1)
x_test = pd.concat([x_test, mon2mon_test.drop(['Unnamed: 0', 'acc_id'], axis = 1)], axis= 1)

### acc_id 제거

In [53]:
x_data = x_data.drop(['acc_id'], axis=1)
x_test = x_test.drop(['acc_id'], axis=1)

In [54]:
len(x_data.columns)

2189

# 트레이닝셋, 테스트 셋 구분
---

In [77]:
x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.5, random_state=0)

In [78]:
x_val, x_test, y_val, y_test = train_test_split(x_val, y_val, test_size = 0.3, random_state = 0)

In [79]:
x_train.shape

(50000, 2189)

In [82]:
logit = 0
result = [0]
result_feature = []

In [99]:
forest = RandomForestClassifier(n_estimators=100, random_state=0, n_jobs=-1, max_depth=25)

In [100]:
forest.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=25, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [101]:
np.mean(f1_score(y_test, forest.predict(x_test), average=None))

0.7316390595269652

## nTree

In [107]:
X = []

In [108]:
for v in forest.estimators_:
    X.append(v.predict(x_val))

In [115]:
from sklearn import linear_model

In [116]:
regr = linear_model.LinearRegression()

In [155]:
regr.fit(X, y_val)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [182]:
regr.predict(X)

array([[ 0.279174  ,  0.28555921,  0.18987528,  0.24539151],
       [ 0.40147753,  0.30693512,  0.25039748,  0.04118986],
       [ 0.35100802,  0.29499771,  0.32140539,  0.03258888],
       ...,
       [ 0.36166661,  0.31136591,  0.2211617 ,  0.10580578],
       [ 0.61307098,  0.45637609,  0.16916075, -0.23860782],
       [ 0.16175622,  0.23342744,  0.30244054,  0.3023758 ]])

In [149]:
X = np.array(X)

In [164]:
X = X.T

In [169]:
X.T

array([[3., 1., 1., ..., 3., 3., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [2., 2., 1., ..., 2., 0., 0.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [2., 2., 1., ..., 1., 0., 2.]])

In [176]:
X = X.T

In [184]:
X[1]

array([1., 1., 1., 1., 1., 1., 0., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1.,
       1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0.,
       1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [162]:
y_val

array([[0, 0, 0, 1],
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       ...,
       [0, 1, 0, 0],
       [1, 0, 0, 0],
       [0, 0, 1, 0]], dtype=uint8)